# Step 1: Merge the data in a single file

**Required Libraries:**
  
  !pip install textblob

  !pip install langdetect

  !pip install torch transformers datasets

In [2]:
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install textblob

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install langdetect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993243 sha256=979c0662c8f03027df3ab823aceb94290576f41564a9824e956b5563104d3aee
  Stored in directory: /root/.cache/pip/wheels/d1/c1/d9/7e068de779d863bc8f8fc9467d85e25cfe47fa5051fff1a1bb
Successfully built langdetect


In [ ]:
!pip install texttable

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
# To mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Import Packages
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
tqdm.pandas()
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import spacy
nltk.download('stopwords')
nltk.download('wordnet')
from textblob import TextBlob
#from langdetect import detect
#from langdetect import LangDetectException

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


## English Review detection steps:

In [ ]:
# Load Course Reviews Data
Course_reviews = pd.read_csv('/content/drive/MyDrive/CMU SUBJECTS/DATA/Course_reviews.csv')

In [ ]:
def detect_language(text):
    try:
        return detect(text)
    except LangDetectException:
        return None

In [ ]:
Course_reviews['lang_detect'] = Course_reviews['reviews'].progress_apply(lambda x: detect_language(str(x)))

In [ ]:
# Only select english reviews and load into csv for further use.
Course_reviews_Eng = Course_reviews[Course_reviews['lang_detect'] == 'en']
Course_reviews_Eng.to_csv('/content/drive/MyDrive/CMU SUBJECTS/DATA/Course_reviews_Eng.csv', index=False)

## Pre Processing the course review data


1.   Remove duplicate rows
2.  Convert word with n't to not
3.  Remove any special character
4. Adding sentiment value and sentiment score for the review 



In [48]:
# Load Course Reviews Data
Course_reviews = pd.read_csv('/content/drive/MyDrive/CMU SUBJECTS/DATA/Clean Data/Course_reviews_Eng.csv')

In [49]:
# Drop duplicates records
print("Before dropping duplicate rows:",len(Course_reviews))
Course_reviews = Course_reviews.drop_duplicates()
print("After dropping duplicate rows:",len(Course_reviews))

Before dropping duplicate rows: 1190399
After dropping duplicate rows: 424417


In [50]:
# Lowercasing 
Course_reviews['clean_reviews']=Course_reviews['reviews'].str.lower()

In [54]:
# Remove Stopwords & Lemmatization:
import spacy
from nltk.corpus import stopwords

nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

def lemmatization_stopwords(text):
    doc = nlp(text)

    # Custom Stop words
    stop_words = set(stopwords.words('english'))
    # Define a list of negation words and their contractions
    negation_words = ["not", "n't", "no", "never", "none", "neither", "nowhere", "nothing",
                    "don't", "doesn't", "didn't", "isn't", "aren't", "wasn't", "weren't",
                    "haven't", "hasn't", "hadn't", "won't", "wouldn't", "can't", "couldn't",
                    "shouldn't"]
        
    # Remove all the negation words and their contractions from stop words
    custom_stop_words = set([word for word in stop_words if word.lower() not in negation_words])
    
    #tokens = [token.text.strip() for token in doc if token.text not in custom_stop_words]# and len(token.text) > 2]
    tokens = [token.lemma_ for token in doc if token.text not in custom_stop_words]
    return " ".join(tokens)

In [55]:
Course_reviews['clean_reviews'] = Course_reviews['clean_reviews'].progress_apply(lemmatization_stopwords)

100%|██████████| 424417/424417 [43:23<00:00, 163.01it/s]


In [56]:
# Removing Non Alphabets characters from review
Course_reviews['clean_reviews'] = Course_reviews['clean_reviews'].progress_apply(lambda x: re.sub(r"n't", " not", x))
Course_reviews['clean_reviews'] = Course_reviews['clean_reviews'].progress_apply(lambda x: re.sub(r'[^a-zA-Z\s.]', ' ', x))

# Remove Whitespace:
def remove_whitespace(text):
    return  " ".join(text.split())
    
Course_reviews["clean_reviews"] = Course_reviews['clean_reviews'].progress_apply(remove_whitespace)

100%|██████████| 424417/424417 [00:01<00:00, 412204.10it/s]


In [1]:
Course_reviews['reviews'].head(10).tolist()

NameError: ignored

In [57]:
Course_reviews['clean_reviews'].head().tolist()

['pretty dry able pass two complete watch be happy . usual question final exam no course annoying far well many microsoft test take . never find suplimental material course reference ... care ... pass',
 'would well experience video screen shot would sho side text instructor go thru user not go way beginning text able view slide instructor show .',
 'information perfect program little annoying . wait minute watch video take quiz . information perfect pass test no issue',
 'grammatical mistake test make double take not bad .',
 'excellent course training provide detailed easy follow .']

In [58]:
# Sentiment Polarity score
def text_polarity_score(text):
    temp_dict = {}
    p = TextBlob(text).sentiment.polarity
    if p >= 0.5:
        label = "Positive"
    elif (p >= 0.0):
        label ="Neutral"
    else:
        label ="Negative"
    temp_dict['label'] = label
    temp_dict['score'] = p
    
    return temp_dict

In [59]:
sentiment_dict = Course_reviews['clean_reviews'].progress_apply(text_polarity_score)

100%|██████████| 424417/424417 [02:00<00:00, 3507.76it/s]


In [60]:
Course_reviews['sentiments'] = [d['label'] for d in sentiment_dict]
Course_reviews['sentiments_score'] = [d['score'] for d in sentiment_dict]

In [62]:
Course_reviews.to_csv('/content/drive/MyDrive/CMU SUBJECTS/DATA/Clean Data/coursera_reviews_clean_data.csv', index=False)

## **Merge all the data:**

*   coursera_courses
*   coursera_review_clean_data
*   University_Global_Rankings
*   Coursera_courses_category





In [63]:
# Load Courses details Data
Coursera_courses = pd.read_csv('/content/drive/MyDrive/CMU SUBJECTS/DATA/Clean Data/Coursera_courses.csv')

In [64]:
# Load Courses details Data
Coursera_reviews = pd.read_csv('/content/drive/MyDrive/CMU SUBJECTS/DATA/Clean Data/coursera_reviews_clean_data.csv')

In [65]:
# Load University_Global_Ranking
University_Global_Ranking = pd.read_csv('/content/drive/MyDrive/CMU SUBJECTS/DATA/Clean Data/University_Global_Ranking.csv')

In [66]:
# Load Coursera_courses__category
Coursera_courses_category = pd.read_csv('/content/drive/MyDrive/CMU SUBJECTS/DATA/Clean Data/Coursera_courses_category.csv',encoding='ISO-8859-1')

In [67]:
# Load the data from csv to temp dataframes
temp_merge_df = pd.merge(Coursera_courses,University_Global_Ranking[["institution","World_Rank","I_Category"]],how="inner",on='institution')
temp_merge_df = pd.merge(temp_merge_df,Coursera_courses_category[["course_id","subject"]],how="inner",on='course_id')
temp_merge_df = pd.merge(temp_merge_df,Coursera_reviews[["course_id","reviews","reviewers","date_reviews","rating","clean_reviews","sentiments","sentiments_score","lang_detect"]],how="inner",on='course_id')

In [37]:
temp_merge_df.head()

,name,institution,course_url,course_id,World_Rank,I_Category,subject,reviews,reviewers,date_reviews,rating,clean_reviews,sentiments,sentiments_score,lang_detect
0,Machine Learning,Stanford University,https://www.coursera.org/learn/machine-learning,machine-learning,3.0,University,Data Science,This is an extremely basic course. Machine lea...,By Deleted A,"Mar 18, 2017",1,this be an extremely basic course . machine le...,Negative,-0.025189,en
1,Machine Learning,Stanford University,https://www.coursera.org/learn/machine-learning,machine-learning,3.0,University,Data Science,The course is ok but the certification procedu...,By Bruno C,"Nov 09, 2015",1,the course be ok but the certification procedu...,Neutral,0.162500,en
2,Machine Learning,Stanford University,https://www.coursera.org/learn/machine-learning,machine-learning,3.0,University,Data Science,"I just started week 3 , I have to admit that I...",By Fadi,"Apr 15, 2019",1,I just start week I have to admit that it be a...,Neutral,0.232000,en
3,Machine Learning,Stanford University,https://www.coursera.org/learn/machine-learning,machine-learning,3.0,University,Data Science,This course is absolute garbage. You get no f...,By Mathew L,"Sep 25, 2015",1,this course be absolute garbage . you get no f...,Negative,-0.183532,en
4,Machine Learning,Stanford University,https://www.coursera.org/learn/machine-learning,machine-learning,3.0,University,Data Science,"However good the material and lectures may be,...",By Rui C,"Dec 12, 2015",1,however good the material and lecture may be t...,Neutral,0.010833,en


In [68]:
temp_merge_df.to_csv('/content/drive/MyDrive/CMU SUBJECTS/DATA/Clean Data/Coursera_Final_Merge_clean_Data.csv', index=False)

## **Hypothesis Testing**

In [ ]:
Final_data = pd.read_csv('/content/drive/MyDrive/CMU SUBJECTS/DATA/Clean Data/Coursera_Final_Merge_clean_Data.csv')

In [ ]:
Final_data.head()

,name,institution,course_url,course_id,World_Rank,I_Category,subject,reviews,reviewers,date_reviews,rating,clean_reviews,sentiments,sentiments_score,lang_detect
0,Machine Learning,Stanford University,https://www.coursera.org/learn/machine-learning,machine-learning,3.0,University,Data Science,This is an extremely basic course. Machine lea...,By Deleted A,"Mar 18, 2017",1,this is an extremely basic course. machine lea...,Neutral,0.045720,en
1,Machine Learning,Stanford University,https://www.coursera.org/learn/machine-learning,machine-learning,3.0,University,Data Science,The course is ok but the certification procedu...,By Bruno C,"Nov 09, 2015",1,the course is ok but the certification procedu...,Neutral,0.162500,en
2,Machine Learning,Stanford University,https://www.coursera.org/learn/machine-learning,machine-learning,3.0,University,Data Science,"I just started week 3 , I have to admit that I...",By Fadi,"Apr 15, 2019",1,i just started week 3 i have to admit that i...,Neutral,0.232000,en
3,Machine Learning,Stanford University,https://www.coursera.org/learn/machine-learning,machine-learning,3.0,University,Data Science,This course is absolute garbage. You get no f...,By Mathew L,"Sep 25, 2015",1,this course is absolute garbage. you get no fe...,Negative,-0.154762,en
4,Machine Learning,Stanford University,https://www.coursera.org/learn/machine-learning,machine-learning,3.0,University,Data Science,"However good the material and lectures may be,...",By Rui C,"Dec 12, 2015",1,however good the material and lectures may be ...,Neutral,0.010833,en


In [ ]:
# Import Statstical package
import statsmodels.api as sm
#from pprint import pprint
#from tabulate import tabulate
#from texttable import Texttable

### **Hypothesis testing between sentiment score and rating**

In [ ]:
pos_neg_reviews = Final_data[Final_data["sentiments"] !="Neutral"]

In [ ]:
X = pos_neg_reviews["sentiments_score"]
Y = pos_neg_reviews['rating']

X = sm.add_constant(X)
model = sm.OLS(Y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                 rating   R-squared:                       0.280
Model:                            OLS   Adj. R-squared:                  0.280
Method:                 Least Squares   F-statistic:                 6.336e+04
Date:                Mon, 17 Apr 2023   Prob (F-statistic):               0.00
Time:                        23:38:18   Log-Likelihood:            -1.5550e+05
No. Observations:              163167   AIC:                         3.110e+05
Df Residuals:                  163165   BIC:                         3.110e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const                3.8991      0.004  

In [ ]:
dummy_vars_i = pd.get_dummies(pos_neg_reviews['I_Category'], prefix='Inst')
dummy_vars_s = pd.get_dummies(pos_neg_reviews['subject'], prefix='sub')
X = pd.concat([pos_neg_reviews["sentiments_score"],dummy_vars_i,dummy_vars_s], axis=1)
Y = pos_neg_reviews['rating']

X = sm.add_constant(X)
model = sm.OLS(Y, X).fit()
summary = model.summary()
print(summary)

                            OLS Regression Results                            
Dep. Variable:                 rating   R-squared:                       0.286
Model:                            OLS   Adj. R-squared:                  0.286
Method:                 Least Squares   F-statistic:                     4671.
Date:                Mon, 17 Apr 2023   Prob (F-statistic):               0.00
Time:                        23:38:27   Log-Likelihood:            -1.5477e+05
No. Observations:              163167   AIC:                         3.096e+05
Df Residuals:                  163152   BIC:                         3.097e+05
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------


In [ ]:
dummy_vars_i = pd.get_dummies(Final_data['I_Category'], prefix='Inst')
dummy_vars_s = pd.get_dummies(Final_data['subject'], prefix='sub')
X = pd.concat([Final_data["sentiments_score"],dummy_vars_i,dummy_vars_s], axis=1)
Y = Final_data['rating']

X = sm.add_constant(X)
model = sm.OLS(Y, X).fit()
summary = model.summary()
print(summary)

                            OLS Regression Results                            
Dep. Variable:                 rating   R-squared:                       0.104
Model:                            OLS   Adj. R-squared:                  0.104
Method:                 Least Squares   F-statistic:                     3529.
Date:                Sat, 15 Apr 2023   Prob (F-statistic):               0.00
Time:                        14:48:34   Log-Likelihood:            -4.4765e+05
No. Observations:              424417   AIC:                         8.953e+05
Df Residuals:                  424402   BIC:                         8.955e+05
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------


In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression

In [ ]:
X = np.array(pos_neg_reviews["sentiments_score"]).reshape((-1,1))
Y = np.array(pos_neg_reviews['rating'])

In [ ]:
X.shape

(162990, 1)

In [ ]:
model = LinearRegression()
model.fit(X, Y)

LinearRegression()

In [ ]:
r_sq = model.score(X, Y)
print(f"coefficient of determination: {r_sq}")

coefficient of determination: 0.2821635293175888
